#SSL+GA



現時点での使い方

・encoder生成

　　import section

　　CL内

　  DataUtils,OtherUtils,Network,Pretrain

　　の順で実行することで作成できます

\

・GA使用法

　　encoderがすでに生成されていれば

　　Pretrain以外順に実行すればできます

　　そのため現在Pre trainの実行部はコメントアウトしており

　　import section,CL,GA part で実行できます

\

参考資料

RAdam:https://github.com/LiyuanLucasLiu/RAdam

SimCLR:https://github.com/leftthomas/SimCLR

#import section

In [ ]:
#gpu 確認
!nvidia-smi

In [ ]:
#drive にデータ保存用
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#https://github.com/LeeDoYup/FixMatch-pytorch/tree/main/datasets

import math
import torch
from torch.utils.data.distributed import DistributedSampler

import torchvision
from torchvision import datasets, transforms
from torch.utils.data import sampler, DataLoader,Dataset
from torchvision.datasets import CIFAR10
from torch.utils.data.sampler import BatchSampler
import torch.distributed as dist
import numpy as np

import copy

import math
import torch.nn as nn
import torch.nn.functional as F

import torchvision.models as models
from torch.cuda.amp import autocast, GradScaler
import os
import contextlib

from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import LambdaLR

import os

import logging
import random
import warnings

import PIL, PIL.ImageOps, PIL.ImageEnhance, PIL.ImageDraw
from PIL import Image

import argparse

import pandas as pd
import torch
import torch.optim as optim
from thop import profile, clever_format
from torch.utils.data import DataLoader
from tqdm import tqdm

import torch.nn as nn
import torch.nn.functional as F
from torchvision.models.resnet import resnet18
from torchvision.models.resnet import resnet50

import time
import datetime

import csv

from deap import base, creator,tools
import shutil
from sklearn.metrics import accuracy_score

In [ ]:
#dset = getattr(torchvision.datasets,"CIFAR10")
#dset = dset("./data", train=True,download=True)
#data, targets = np.array(dset.data), np.array(dset.targets)

Files already downloaded and verified


#Contrastive Learning

##Data Utils

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

valid_transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

#
class CIFAR10Pair(CIFAR10):
    """CIFAR10 Dataset.
    """

    def __getitem__(self, index):
        img, target = self.data[index], self.targets[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            pos_1 = self.transform(img)
            pos_2 = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return pos_1, pos_2, target


In [ ]:
"""
ラベル分割
(data,target):(入力データ,ラベル)
num_labels:ラベル付きデータの枚数
num_classes:ラベル数
index:ラベル付きデータが確定しているときのインデックス
include_lb_to_ulb:ラベルなしにもラベル付き入力データを含ませるか否か
"""
def split_ga_data(data, target, num_labels, num_classes, index=None, include_lb_to_ulb=True):
    data, target = np.array(data), np.array(target)
    lb_data, lbs, lb_idx = sample_labeled_data(data, target, num_labels, num_classes, index)
    ulb_idx = np.array(sorted(list(set(range(len(data))) - set(lb_idx)))) #unlabeled_data index of data
    if include_lb_to_ulb:
        return lb_data, lbs, data, target
    else:
        return lb_data, lbs, data[ulb_idx], target[ulb_idx]
    
    
"""
データ選出
(data,target):(入力データ,ラベル)
num_labels:ラベル付きデータの枚数
num_classes:ラベル数
index:ラベル付きデータが確定しているときのインデックス
isUniform:ラベルに対するデータを均一にとるか否か
"""
def sample_labeled_data(data, target, 
                         num_labels,
                         num_classes,
                         index=None,
                        isUniform=True):
    #assert num_labels % num_classes == 0
    if not index is None:
        index = np.array(index, dtype=np.int32)
        return data[index], target[index], index
    
    samples_per_class = int(num_labels / num_classes)
    
    lb_data = []
    lbs = []
    lb_idx = []
    if isUniform:
        for c in range(num_classes):
            idx = np.where(target == c)[0]
            idx = np.random.choice(idx, samples_per_class, False)
            lb_idx.extend(idx)
            lb_data.extend(data[idx])
            lbs.extend(target[idx])
    else:
        idx = np.random.choice(target, num_labels, False)
        lb_idx.extend(idx)
        lb_data.extend(data[idx])
        lbs.extend(target[idx])
        
    return np.array(lb_data), np.array(lbs), np.array(lb_idx)


In [ ]:
"""
基本のデータセット
"""
class BasicDataset(Dataset):
    def __init__(self,
                 data,
                 targets=None,
                 num_classes=None,
                 transform=None,
                 onehot=False,
                 *args, **kwargs):

        super(BasicDataset, self).__init__()
        self.data = data
        self.targets = targets
        
        self.num_classes = num_classes
        self.onehot = onehot
        
        self.transform = transform
        self.target_transform = None
                
    
    def __getitem__(self, index):
        img, target = self.data[index], self.targets[index]
        img = Image.fromarray(img)

        return self.transform(img), target

    
    def __len__(self):
        return len(self.data)

In [ ]:
"""
GA用データセット
"""
class GA_Dataset:
    
    def __init__(self,
                 name='cifar10',
                 train=True,
                 num_classes=10,
                 data_dir='./data'):
        
        self.name = name
        self.train = train
        self.data_dir = data_dir
        self.num_classes = num_classes

        if self.train is True:
            self.transform = train_transform
        else:
            self.transform = test_transform
        
    def get_data(self):
        
        dset = getattr(torchvision.datasets, self.name.upper())
        dset = dset(self.data_dir, train=self.train, download=True)
        data, targets = dset.data, dset.targets
        return data, targets
    
    
    def get_dset(self, use_strong_transform=False, 
                 strong_transform=None, onehot=False):
        
        data, targets = self.get_data()
        num_classes = self.num_classes
        transform = self.transform
        data_dir = self.data_dir
        
        return BasicDataset(data, targets, num_classes, transform, onehot)
    

In [ ]:
"""
データ生成(正確にはインデックスに応じた分割)
train_index:ラベル付きデータのインデックス
temp_index:ラベルをつけたいデータのインデックス
temp_target:GAの1個体であるラベル集合
eval_index:テストデータのインデックス
"""
def creat_dset(train_index,temp_index,temp_target,eval_index,
               name='cifar10',num_classes=10,data_dir='./data',train=True):
    dset = getattr(torchvision.datasets, name.upper())
    dset = dset(data_dir, train=train, download=False)
    data, targets = np.array(dset.data), np.array(dset.targets)

    lb_data, lbs, lb_idx = sample_labeled_data(data, targets,
                                               len(train_index), num_classes, train_index)
    if len(temp_target) != 0:
      te_data, tes, te_idx = sample_labeled_data(data, targets, 
                                               len(temp_index), num_classes, temp_index)
      lb_data = np.concatenate([lb_data, np.array(te_data)])
      lbs = np.concatenate([lbs,np.array(temp_target)])

    ev_data, evs, ev_idx = sample_labeled_data(data, targets, 
                                               len(eval_index), num_classes, eval_index)
    return lb_data, lbs, ev_data, evs
"""
データセット生成
"""
def creat_GA_Dataset(train_index,temp_index,temp_target,eval_index,
                      name='cifar10',num_classes=10,data_dir='./data'):
    lb_data,lb_targets,ev_data,ev_targets = creat_dset(train_index,temp_index,temp_target,eval_index,
                                                                          name,num_classes,data_dir,train=True)
    onehot =False
    lb_dset = BasicDataset(lb_data, lb_targets, num_classes, 
                               train_transform, onehot)
    ev_dset = BasicDataset(ev_data, ev_targets, num_classes,
                           valid_transform, onehot)
    
    return lb_dset,  ev_dset

"""
上記に加え
train dataにランダムなラベルが付与せれたダミーデータを追加
"""
def creat_pDMdset(train_index,temp_index,temp_target,eval_index,dum_index,dum_target,
               name='cifar10',num_classes=10,data_dir='./data',train=True):
    dset = getattr(torchvision.datasets, name.upper())
    dset = dset(data_dir, train=train, download=False)
    data, targets = np.array(dset.data), np.array(dset.targets)

    lb_data, lbs, lb_idx = sample_labeled_data(data, targets,
                                               len(train_index), num_classes, train_index)
    if len(temp_target) != 0:
      te_data, tes, te_idx = sample_labeled_data(data, targets, 
                                               len(temp_index), num_classes, temp_index)
      lb_data = np.concatenate([lb_data, np.array(te_data)])
      lbs = np.concatenate([lbs,np.array(temp_target)])
    
    if len(dum_target) != 0:
      dum_data, dums, dum_idx = sample_labeled_data(data, targets, 
                                               len(dum_index), num_classes, dum_index)
      lb_data = np.concatenate([lb_data, np.array(dum_data)])
      lbs = np.concatenate([lbs,np.array(dum_target)])

    ev_data, evs, ev_idx = sample_labeled_data(data, targets, 
                                               len(eval_index), num_classes, eval_index)
    return lb_data, lbs, ev_data, evs

def creat_GADM_Dataset(train_index,temp_index,temp_target,eval_index,dum_index=None,dum_target=None,
                      name='cifar10',num_classes=10,data_dir='./data'):
    lb_data,lb_targets,ev_data,ev_targets = creat_pDMdset(train_index,temp_index,temp_target,eval_index,
                                                         dum_index,dum_target,name,num_classes,data_dir,train=True)
    #train_trans = get_transform(mean[name], std[name], train=True)
    #eval_trans = get_transform(mean[name], std[name], train=False)

    onehot =False
    lb_dset = BasicDataset(lb_data, lb_targets, num_classes, 
                               train_transform, onehot)
    ev_dset = BasicDataset(ev_data, ev_targets, num_classes,
                           valid_transform, onehot)
    
    return lb_dset,  ev_dset


##Other Utils

In [ ]:
import math
import torch
from torch.optim.optimizer import Optimizer, required

"""
RAdamというoptimizer
初期学習に自動化されたWarmUpSGD、途中からAdam
のようなイメージ
引用元:https://github.com/LiyuanLucasLiu/RAdam
"""
class RAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0, degenerated_to_sgd=True):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        
        self.degenerated_to_sgd = degenerated_to_sgd
        if isinstance(params, (list, tuple)) and len(params) > 0 and isinstance(params[0], dict):
            for param in params:
                if 'betas' in param and (param['betas'][0] != betas[0] or param['betas'][1] != betas[1]):
                    param['buffer'] = [[None, None, None] for _ in range(10)]
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, buffer=[[None, None, None] for _ in range(10)])
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = group['buffer'][int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    elif self.degenerated_to_sgd:
                        step_size = 1.0 / (1 - beta1 ** state['step'])
                    else:
                        step_size = -1
                    buffered[2] = step_size

                # more conservative since it's an approximated value
                if N_sma >= 5:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size * group['lr'], exp_avg, denom)
                    p.data.copy_(p_data_fp32)
                elif step_size > 0:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    p_data_fp32.add_(-step_size * group['lr'], exp_avg)
                    p.data.copy_(p_data_fp32)

        return loss

In [ ]:
"""
WarmUpつきcosスケジューラ
"""
def get_lr(step, total_steps, lr_max, lr_min):
    """Compute learning rate according to cosine annealing schedule."""
    return lr_min + (lr_max - lr_min) * 0.5 * (1 + np.cos(step / total_steps * np.pi))

def get_cosine_schedule_with_warmup(optimizer,
                                    num_training_steps,
                                    num_cycles=7./16.,
                                    num_warmup_steps=0,
                                    last_epoch=-1):
    '''
    Get cosine scheduler (LambdaLR).
    if warmup is needed, set num_warmup_steps (int) > 0.
    '''
    
    def _lr_lambda(current_step):
        '''
        _lr_lambda returns a multiplicative factor given an interger parameter epochs.
        Decaying criteria: last_epoch
        '''
        
        if current_step < num_warmup_steps:
            _lr = float(current_step) / float(max(1, num_warmup_steps))
        else:
            num_cos_steps = float(current_step - num_warmup_steps)
            num_cos_steps = num_cos_steps / float(max(1, num_training_steps - num_warmup_steps))
            _lr = max(0.0, math.cos(math.pi * num_cycles * num_cos_steps))
        return _lr
    
    return LambdaLR(optimizer, _lr_lambda, last_epoch)

##Network

In [ ]:

"""
事前学習(encoder + projection head)
"""
class Model(nn.Module):
    def __init__(self,base_encoder=resnet18(pretrained=False), projection_dim=128):
        super(Model, self).__init__()

        self.f = []
        for name, module in resnet18().named_children():
            if name == 'conv1':
                module = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
            if not isinstance(module, nn.Linear) and not isinstance(module, nn.MaxPool2d):
                self.f.append(module)
        # encoder
        self.f = nn.Sequential(*self.f)

        # projection head
        self.g = nn.Sequential(nn.Linear(512, 1024, bias=False), nn.BatchNorm1d(1024),
                               nn.ReLU(inplace=True), nn.Linear(1024, projection_dim, bias=True))

    def forward(self, x):
        x = self.f(x)
        feature = torch.flatten(x, start_dim=1)
        out = self.g(feature)
        return F.normalize(feature, dim=-1), F.normalize(out, dim=-1)

"""
分類器学習(encoder + classifier)
encoder の出力次元とclassifierの入力次元をあわせること
"""
class Net(nn.Module):
    def __init__(self, num_classes, pretrained_path):
        super(Net, self).__init__()

        # encoder
        self.f = Model().f
        # classifier
        self.fc = nn.Linear(512, num_classes, bias=True)
        self.load_state_dict(torch.load(pretrained_path, map_location='cpu'), strict=False)

    def forward(self, x):
        x = self.f(x)
        feature = torch.flatten(x, start_dim=1)
        out = self.fc(feature)
        return out


##Pre Train

In [ ]:
"""
encoderの学習クラス
net:ネットワーク
feature_dim:出力次元
temperature:Contrastive Learning(: CL)の温度係数
batch_size,epochs:バッチサイズ,エポック数
save_dir:保存フォルダ

k,cについて
test dataに対してtrain dataのc枚におけるfeatureが似ているものをk枚取り出して
それらのラベルの割合からtest dataのラベルを推定してtestの識別率の数値を出している

ラベル数が限られている実験なので不必要
一応残しているが消したほうがいいかも
"""
class  CL_Train:
    def __init__(self, net, feature_dim, temperature, k, batch_size, epochs, save_dir, c):

        super(CL_Train, self).__init__()

        self.net = net
        self.feature_dim = feature_dim
        self.temperature = temperature
        self.k = k
        self.c = c
        
        self.batch_size = batch_size
        self.epochs = epochs

        self.optimizer = None
        self.loader_dict = {}

        self.save_dir = save_dir

        self.it = 0
        self.best_acc = 0


    def set_data_loader(self, loader_dict):
        self.loader_dict =loader_dict
    
    def set_optimizer(self, optimizer, scheduler=None):
        self.optimizer = optimizer
        if scheduler is not None:
            self.scheduler = scheduler
        else:
            self.scheduler = None

    def train_1epoch(self):
        self.net.train()
        total_loss, total_num = 0.0, 0
        for pos_1, pos_2, target in self.loader_dict['train']:
            pos_1, pos_2 = pos_1.cuda(non_blocking=True), pos_2.cuda(non_blocking=True)
            feature_1, out_1 = self.net(pos_1)
            feature_2, out_2 = self.net(pos_2)
            # [2*B, D]
            out = torch.cat([out_1, out_2], dim=0)
            # [2*B, 2*B]
            sim_matrix = torch.exp(torch.mm(out, out.t().contiguous()) / self.temperature)
            mask = (torch.ones_like(sim_matrix) - torch.eye(2 * self.batch_size, device=sim_matrix.device)).bool()
            # [2*B, 2*B-1]
            sim_matrix = sim_matrix.masked_select(mask).view(2 * self.batch_size, -1)

            # compute loss
            pos_sim = torch.exp(torch.sum(out_1 * out_2, dim=-1) / self.temperature)
            # [2*B]
            pos_sim = torch.cat([pos_sim, pos_sim], dim=0)
            loss = (- torch.log(pos_sim / sim_matrix.sum(dim=-1))).mean()
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            if self.scheduler:
                self.scheduler.step()

            total_num += self.batch_size
            total_loss += loss.item() * self.batch_size

        return total_loss / total_num


    def test(self):
        self.net.eval()
        total_top1, total_top5, total_num, feature_bank = 0.0, 0.0, 0, []
        with torch.no_grad():
            # generate feature bank
            for data, _, target in self.loader_dict['memory']:
                feature, out = self.net(data.cuda(non_blocking=True))
                feature_bank.append(feature)
            # [D, N]
            feature_bank = torch.cat(feature_bank, dim=0).t().contiguous()
            # [N]
            feature_labels = torch.tensor(self.loader_dict['memory'].dataset.targets, device=feature_bank.device)
            
            for data, _, target in self.loader_dict['test']:
                data, target = data.cuda(non_blocking=True), target.cuda(non_blocking=True)
                feature, out = self.net(data)

                total_num += data.size(0)
                # compute cos similarity between each feature vector and feature bank ---> [B, N]
                sim_matrix = torch.mm(feature, feature_bank)
                # [B, K]
                sim_weight, sim_indices = sim_matrix.topk(k=self.k, dim=-1)
                # [B, K]
                sim_labels = torch.gather(feature_labels.expand(data.size(0), -1), dim=-1, index=sim_indices)
                sim_weight = (sim_weight / self.temperature).exp()

                # counts for each class
                one_hot_label = torch.zeros(data.size(0) * self.k, self.c, device=sim_labels.device)
                # [B*K, C]
                one_hot_label = one_hot_label.scatter(dim=-1, index=sim_labels.view(-1, 1), value=1.0)
                # weighted score ---> [B, C]
                pred_scores = torch.sum(one_hot_label.view(data.size(0), -1, self.c) * sim_weight.unsqueeze(dim=-1), dim=1)

                pred_labels = pred_scores.argsort(dim=-1, descending=True)
                total_top1 += torch.sum((pred_labels[:, :1] == target.unsqueeze(dim=-1)).any(dim=-1).float()).item()
                total_top5 += torch.sum((pred_labels[:, :5] == target.unsqueeze(dim=-1)).any(dim=-1).float()).item()

        return total_top1 / total_num, total_top5 / total_num


    def train(self):
        results = {'train_loss': [], 'test_acc@1': [], 'test_acc@5': []}
        save_name_pre = '{}_{}_{}_{}_{}'.format(self.feature_dim, self.temperature,\
                                                self.k, self.batch_size, self.epochs)
        os.makedirs(self.save_dir,exist_ok=True)
        if self.it is 0:
            with open('{}{}_statistics.csv'.format(self.save_dir,save_name_pre), 'w') as f:
                writer = csv.writer(f)
                writer.writerow(['epoch', 'train_loss', 'test_acc@1', 'test_acc@5'])
        best_acc = self.best_acc
        start = self.it + 1
        
        print('[{}]//Start Epoch:{}'.format(datetime.datetime.now(), start))
        for epoch in range(start, self.epochs + 1):
            train_loss = self.train_1epoch()
            results['train_loss'].append(train_loss)
            test_acc_1, test_acc_5 = self.test()
            results['test_acc@1'].append(test_acc_1)
            results['test_acc@5'].append(test_acc_5)
            print('[{}]//Test Epoch: [{}/{}] Loss:{:.2f} Acc@1:{:.2f} Acc@5:{:.2f}'
                                     .format(datetime.datetime.now(), epoch, self.epochs, \
                                             train_loss, test_acc_1, test_acc_5))
            #data_frame = pd.DataFrame(data=results, index=range(1, epoch + 1))
            #data_frame.to_csv('{}{}_statistics.csv'.format(self.save_dir,save_name_pre), index_label='epoch')
            self.it = epoch
            if test_acc_1 >= best_acc:
                best_acc = test_acc_1
                torch.save(self.net.state_dict(), '{}best_model.pth'.format(args.save_dir))
                print('best model saved')
            self.save_state( 'state.pth', args.save_dir,best_acc)
            torch.save(self.net.state_dict(), '{}latest_model.pth'.format(args.save_dir))
            with open('{}{}_statistics.csv'.format(self.save_dir,save_name_pre), 'a') as f:
                writer = csv.writer(f)
                writer.writerow([epoch, train_loss, test_acc_1, test_acc_5])


    def save_model(self, save_name, save_path):
        save_filename = os.path.join(save_path, save_name)
        torch.save({'net':self.net.state_dict(),
                    'optimizer': self.optimizer.state_dict(), #'scheduler':self.scheduler.state_dict(),
                    'it': self.it}, save_filename)
        
    def save_state(self, save_name, save_path,best_acc):
        save_filename = os.path.join(save_path, save_name)
        if self.scheduler is None:
            torch.save({'best_acc':best_acc,
                      'optimizer': self.optimizer.state_dict(),
                      'it': self.it}, save_filename)
        else:
            torch.save({'best_acc':best_acc,
                      'optimizer': self.optimizer.state_dict(),
                      'scheduler':self.scheduler.state_dict(),
                      'it': self.it}, save_filename)
    
    
    def load_model(self, load_path):
        checkpoint = torch.load(load_path)

        for key in checkpoint.keys():
            #if hasattr(self, key) and getattr(self, key) is not None:
            if 'net' in key:
                self.net.load_state_dict(checkpoint[key], strict=False)
            elif 'it' in key:
                #self.it = 0
                self.it = checkpoint[key]
            elif 'optimizer' in key:
                self.optimizer.load_state_dict(checkpoint[key])
            elif 'scheduler' in key:
                self.scheduler.load_state_dict(checkpoint[key])
            elif 'best_acc'  in  key:
                self.best_acc  =checkpoint[key]
            else:
                getattr(self, key).load_state_dict(checkpoint[key])
            print(f"Check Point Loading: {key} is LOADED")


In [ ]:
"""
事前学習を実行
"""
def CL_main(args):
    
    train_data = CIFAR10Pair(root='data', train=True, transform=train_transform, download=True)
    train_loader = DataLoader(train_data, batch_size=args.batch_size, shuffle=True, num_workers=1, pin_memory=True,
                              drop_last=True)
    memory_data = CIFAR10Pair(root='data', train=True, transform=test_transform, download=True)
    memory_loader = DataLoader(memory_data, batch_size=args.batch_size, shuffle=False, num_workers=1, pin_memory=True)
    test_data = CIFAR10Pair(root='data', train=False, transform=test_transform, download=True)
    test_loader = DataLoader(test_data, batch_size=args.batch_size, shuffle=False, num_workers=1, pin_memory=True)
    loader_dict = {'train':train_loader,'memory':memory_loader,'test':test_loader}

    model = Model(projection_dim=args.projection_dim).cuda()
    flops, params = profile(model, inputs=(torch.randn(1, 3, 32, 32).cuda(),))
    flops, params = clever_format([flops, params])
    print('# Model Params: {} FLOPs: {}'.format(params, flops))
    c = len(memory_data.classes)

    trainer = CL_Train(model,
                       args.projection_dim,
                       args.temperature,
                       args.k,
                       args.batch_size,
                       args.epochs,
                       args.save_dir,
                       c)
    optimizer = RAdam(params=model.parameters(), lr=args.lr)
    scheduler = LambdaLR(
        optimizer,
        lr_lambda=lambda step: get_lr(  
            step,
            args.epochs * len(train_loader),
            args.lr,  # lr_lambda computes multiplicative factor
            1e-6))
    trainer.set_optimizer(optimizer)
    trainer.set_data_loader(loader_dict)

    if os.path.exists(args.load_path) and args.resume:
        trainer.load_model(args.save_dir+'state.pth')
        trainer.net.load_state_dict(torch.load(args.load_path),strict=False)
    #trainer.scheduler = None
    #trainer.set_optimizer(optimizer)
    
    trainer.train()

In [ ]:
#パラメータ設定
import easydict
def get_pretrain_args(main_path):
    args = easydict.EasyDict({
        'projection_dim':128, 'temperature':0.5, 'k':200,
        'batch_size':1024, 'epochs':1000,

        'lr':1e-6, 'weight_decay':1e-6,
        
        'seed':0,  'resume':True,
        
         'save_dir':main_path+'pretrain/',
         'load_path':main_path+'pretrain/latest_model.pth'

    })
    return args

In [ ]:
"""
保存フォルダおよびパラメータの設定をして
事前学習を実行
実験が途切れて途中からでもok
"""

"""
main_path = '/content/drive/My Drive/CL3/'
args = get_pretrain_args(main_path)
#args.batch_size = 216
print(datetime.datetime.now())
CL_main(args)
"""

##Train 'n' Eval

In [ ]:
"""
classifier の学習
test_times:test dataを水増する倍率
(test dataがラベル付きデータのため非常に少ないことを想定して)
"""
class FineTuning:
    def __init__(self, net, num_classes,batch_size, epochs,save_dir, is_save,test_times):

        super(FineTuning, self).__init__()

        self.net = net
        
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.epochs = epochs

        self.optimizer = None
        self.loader_dict = {}

        self.save_dir = save_dir

        self.it = 0

        self.is_save = is_save

        self.test_times = 1
        if test_times is not None:
          self.test_times = test_times

    def set_data_loader(self, loader_dict):
        self.loader_dict =loader_dict

    def set_optimizer(self, optimizer):
        self.optimizer = optimizer

    def set_loss(self, loss=None):
        if loss is None:
            self.loss = nn.CrossEntropyLoss()
        else:
            self.loss = loss

    def train_val(self, is_train):
        if is_train:
            self.net.train()
            data_loader = self.loader_dict['train']
        else:
            self.net.eval()
            data_loader = self.loader_dict['test']

        total_loss, total_correct_1, total_num = 0.0, 0.0, 0
        with (torch.enable_grad() if is_train else torch.no_grad()):
            for data, target in data_loader:
                data, target = data.cuda(non_blocking=True), target.cuda(non_blocking=True)
                out = self.net(data)
                loss = self.loss(out, target)

                if is_train:
                    self.optimizer.zero_grad()
                    loss.backward()
                    self.optimizer.step()

                total_num += data.size(0)
                total_loss += loss.item() * data.size(0)
                prediction = torch.argsort(out, dim=-1, descending=True)
                total_correct_1 += torch.sum((prediction[:, 0:1] == target.unsqueeze(dim=-1)).any(dim=-1).float()).item()
                
        return total_loss / total_num, total_correct_1 / total_num

    def train(self):
        os.makedirs(self.save_dir,exist_ok=True)
        results = {'train_loss': [], 'train_acc@1': [],
                    'test_loss': [], 'test_acc@1': []}
        best_acc = 0.0
        start = self.it + 1
        
        for epoch in range(start, self.epochs + 1):
            train_loss, train_acc_1 = self.train_val(is_train=True)
            results['train_loss'].append(train_loss)
            results['train_acc@1'].append(train_acc_1)
            test_loss, test_acc_1 = 0,0
            for _ in range(self.test_times):
              t_loss,t_acc_1 = self.train_val(is_train=False)
              test_loss += t_loss
              test_acc_1 += t_acc_1
            test_loss /= self.test_times
            test_acc_1 /= self.test_times
            results['test_loss'].append(test_loss)
            results['test_acc@1'].append(test_acc_1)

            with open('{}res.csv'.format(self.save_dir), 'a') as f:
                writer = csv.writer(f)
                writer.writerow([epoch, train_loss, train_acc_1, test_loss, test_acc_1])
            print('[{}]//Test Epoch: [{}/{}] TrainLoss:{:.2f} TrainAcc@1:{:.2f} TestLoss:{:.2f} TestAcc@1:{:.2f}'
                                     .format(datetime.datetime.now(), epoch, self.epochs, \
                                             train_loss,train_acc_1, test_loss, test_acc_1))
            self.it = epoch
            if test_acc_1 >= best_acc:
                best_acc = test_acc_1
                if self.is_save is True:
                    self.save_classifier('best_fc_model.pth', os.path.join(self.save_dir))

        return results['train_acc@1'][-1], best_acc, test_acc_1

    def save_classifier(self, save_name, save_path):
        save_filename = os.path.join(save_path, save_name)

        torch.save({'classifier': self.net.fc.state_dict(),
                    'optimizer': self.optimizer.state_dict(),#'scheduler': self.scheduler.state_dict(),
                    'it': self.it}, save_filename)
    
    
    def load_classifer(self, load_path):
        checkpoint = torch.load(load_path)

        for key in checkpoint.keys():
            if 'classifier' in key:
                self.net.fc.load_state_dict(checkpoint[key], strict=False)
            elif key == 'it':
                #self.it = 0
                self.it = checkpoint[key]
            elif key == 'optimizer':
                self.optimizer.load_state_dict(checkpoint[key])
            else:
                getattr(self, key).load_state_dict(checkpoint[key])
            print(f"Check Point Loading: {key} is LOADED")


"\n    train_data = CIFAR10(root='data', train=True, transform=train_transform, download=True)\n    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)\n    test_data = CIFAR10(root='data', train=False, transform=test_transform, download=True)\n    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=16, pin_memory=True)\n    loader_dict = {'train':train_loader, 'test':test_loader}\n"

In [ ]:
#パラメータ設定
import easydict
def get_tuning_args(main_path):
    args = easydict.EasyDict({
        'batch_size':64, 'epochs':100,
        'num_classes':10, 'dataset':'cifar10',

        'lr':5e-3, 'weight_decay':1e-6,
        
        'seed':0,  'resume':False,
        'is_save':False, 'is_ga':True,
        'test_times':1,
        
        'index_dir':'none',
        'data_dir':'./data',

        'save_dir':main_path+'fineTuning/',
        'load_encoder_path':main_path+'pretrain/latest_model.pth',
        'load_fc_path':main_path+'fineTuning/best_fc_model.pth'

    })
    return args

In [ ]:
"""
classifier学習の実行
"""
def tuning_main(args,lb_idx,val_idx,temp_idx = [],ind = [],dum_tar = []):

    dum_idx = None
    if len(dum_tar) is not 0:
        dum_idx = np.load(args.index_dir + "dummy.npy")

    if args.is_ga:
        train_dset,eval_dset = creat_GADM_Dataset(train_index=lb_idx, temp_index=temp_idx,
                                                temp_target=ind, eval_index=val_idx,
                                                dum_index=dum_idx, dum_target=dum_tar,
                                                name=args.dataset,num_classes=args.num_classes,
                                                data_dir=args.data_dir)
    else:
        _train_dset = GA_Dataset(name=args.dataset, train=True, 
                             num_classes=args.num_classes, data_dir=args.data_dir)
        train_dset = _train_dset.get_dset()

    if args.use_test:
        _eval_dset = GA_Dataset(name=args.dataset, train=False, 
                             num_classes=args.num_classes, data_dir=args.data_dir)
        eval_dset = _eval_dset.get_dset()

    train_loader = DataLoader(train_dset, batch_size=args.batch_size, shuffle=True, num_workers=1, pin_memory=True)
    test_loader = DataLoader(eval_dset, batch_size=args.batch_size, shuffle=False, num_workers=1, pin_memory=True)
    loader_dict = {'train':train_loader, 'test':test_loader}

    model = Net(num_classes=args.num_classes, pretrained_path=args.load_encoder_path).cuda()
    for param in model.f.parameters():
        param.requires_grad = False
  
    trainer = FineTuning(model,
                         args.num_classes,
                         args.batch_size,
                         args.epochs,
                         args.save_dir,
                         args.is_save,
                         args.test_times)
    
    optimizer = RAdam(params=model.parameters(), lr=args.lr)
    trainer.set_optimizer(optimizer)
    trainer.set_loss()
    trainer.set_data_loader(loader_dict)

    if os.path.exists(args.load_fc_path) and args.resume:
        trainer.load_model(args.load_fc_path)
    
    train_acc, test_acc, last_acc = trainer.train()
    
    return train_acc,test_acc, last_acc

In [ ]:
"""
あるデータに対する予測
"""
def eval_(args,index_):
    checkpoint = torch.load(args.load_fc_path)
    net = Net(num_classes=args.num_classes, pretrained_path=args.load_encoder_path).cuda()
    
    net.fc.load_state_dict(checkpoint['classifier'])
    
    if torch.cuda.is_available():
        net.cuda()
    net.eval()

    val_idx = index_
    temp_idx = np.load(args.index_dir + "temp.npy")

    dset = getattr(torchvision.datasets, "CIFAR10")
    dset = dset(args.data_dir, train=True, download=False)
    data, targets = np.array(dset.data), np.array(dset.targets)

    ev_data, evs, ev_idx = sample_labeled_data(data, targets, 
                                               len(val_idx), args.num_classes, val_idx)
    te_data, tes, te_idx = sample_labeled_data(data, targets, 
                                               len(temp_idx), args.num_classes, temp_idx)
    
    #eval_trans = get_transform(mean["cifar10"], std["cifar10"], train=False)
    ev_dset = BasicDataset(ev_data, evs.tolist(), args.num_classes, test_transform, 
                              False, None, False)
    te_dset = BasicDataset(te_data, tes.tolist(), args.num_classes, test_transform, 
                              False, None, False)
    eval_loader = DataLoader(ev_dset, batch_size=args.batch_size,
                              shuffle=False, num_workers=1, pin_memory=True)
    temp_loader = DataLoader(te_dset, batch_size=args.batch_size,
                              shuffle=False, num_workers=1, pin_memory=True)
 
    acc = 0.0
    ind = []
    ind2 = []
    device2 = torch.device('cpu')
    with torch.no_grad():
        for image, target in eval_loader:
            image = image.type(torch.FloatTensor).cuda()
            logits = net(image)
            acc += logits.cpu().max(1)[1].eq(target).sum().numpy()

        for image, target in temp_loader:
            image = image.type(torch.FloatTensor).cuda()
            logits = net(image)
            pseudo_label = torch.softmax(logits, dim=-1)
            max_probs, max_idx = torch.max(pseudo_label, dim=-1)
            p_label = max_idx.to(device2).detach().clone().numpy().tolist()
            ind += p_label
            ind2 += pseudo_label.to(device2).detach().clone().numpy().tolist()
    
    return acc/len(ev_dset),ind, ind2

#GA Part

##GA Utils

In [ ]:
#個体学習のパラメータ設定
import easydict
def get_args(main_path,g,i):
    args = get_tuning_args(main_path)

    args.save_dir = main_path + "{:03}gene/fix/{:03}/".format(g,i)
    args.index_dir = main_path + 'data_index/'
    args.use_test = False

    return args

def get_eval_args(main_path):
    args = easydict.EasyDict({
        'load_path':main_path + 'base/' + 'fixmatch/model_best.pth',
        'use_train_model':'store_true',

        'net':'WideResNet','net_from_name':False,'depth':16,
        'widen_factor':2,'leaky_slope':0.1,'dropout':0.0,

        'batch_size':100,'data_dir':'./data','dataset':'cifar10',
        'num_classes':10,'index_dir':main_path + 'data_index/'

    })
    return args


IND_SIZE = 100                    #遺伝子数(ラベルを探索するデータ数)
GENERATION_SIZE = 50              #1世代あたりの個体数
#max_ = 10                        #ラベル数
#max_Magnitude = max_ - 1         #遺伝子の最大値

LABELED_SIZE = 0                  #ラベル付きデータ数(適応度計算時にtrainとして使用)
VALIDATION_SIZE = 50              #ラベル付きデータ数(適応度計算時にtestとして使用)
DUM_SIZE = 200                    #ダミーデータ数

gene_his = 'gene_his.csv'         #適応度保存ファイル名
pre_path = 'fineTuning/'            #classifierの事前学習のフォルダ名

#交叉率,個体ごとの突然変異率,遺伝子座ごとの突然変異率,世代数
CXPB, MUTPB, MUTPB2, NGEN = 0.9, 0.5, 0.1, 250
num_trial = 1                     #適応度計算回数
elite_num =2                      #エリート個体数

#TDGAパラメータ(最終項目参照)
t_init = 9e-3
t_fin = 1e-5
eps = 1e-7


In [ ]:
"""
データの分割
ラベル付きデータ/ラベルを探索するデータ/テストデータ
のインデックスをnumpyで保存
"""
def split_GAdata(main_path,num_lb,num_val,num_prov):
    dset = getattr(torchvision.datasets, "cifar10".upper())
    dset = dset("./data", train=True)
    data, targets = dset.data, dset.targets
    data, targets = np.array(data), np.array(targets)

    #使用データのサンプリング
    tmp_data,tmp_s,tmp_idx = sample_labeled_data(data, targets, 
                         num_labels= num_lb + num_val + num_prov ,num_classes=10,
                         index=None,isUniform = True)
    
    #ラベル探索データとラベル付きデータの分割
    prov_data,prov_s,prov_idx = sample_labeled_data(data[tmp_idx], targets[tmp_idx], 
                         num_labels= num_prov ,num_classes=10,
                         index=None, isUniform=True)
    lb_all_idx = np.array(sorted(list(set(range(len(tmp_data))) - set(prov_idx))))
    prov_idx = tmp_idx[prov_idx.tolist()]
    lb_all_idx = tmp_idx[lb_all_idx.tolist()]

    #trainに組み込むものとvalidに使用するものに分割
    lb_data,lb_s,lb_idx = sample_labeled_data(data[lb_all_idx], targets[lb_all_idx], 
                         num_labels= num_lb ,num_classes=10,
                         index=None,isUniform=True)
    val_idx = np.array(sorted(list(set(range(len(lb_all_idx))) - set(lb_idx))))
    lb_idx = lb_all_idx[lb_idx.tolist()]
    val_idx = lb_all_idx[val_idx.tolist()]

    os.makedirs(main_path+"data_index",exist_ok=True)
    np.save(main_path+"data_index/lb",lb_idx)
    np.save(main_path+"data_index/lb_init",lb_idx)
    np.save(main_path+"data_index/val",val_idx)
    np.save(main_path+"data_index/temp",prov_idx)
    np.save(main_path+"data_index/temp_all",prov_idx)
    np.save(main_path+"data_index/lb_all",lb_all_idx)
    np.save(main_path+"data_index/correct",prov_s)
    #return tmp_idx,tmp_s,prov_idx,prov_s,lb_idx,lb_s,val_idx#,val_s


#上記に加えダミーデータも分割するもの
def split_GADMdata(main_path,num_lb,num_val,num_prov,num_dum):
    dset = getattr(torchvision.datasets, "cifar10".upper())
    dset = dset("./data", train=True)
    data, targets = dset.data, dset.targets
    data, targets = np.array(data), np.array(targets)

    #使用データのサンプリング
    tmp_data,tmp_s,tmp_idx = sample_labeled_data(data, targets, 
                         num_labels= num_lb + num_val + num_prov + num_dum ,num_classes=10,index=None)
    
    #ダミー用データのサンプリング
    ul_data,ul_s,ul_idx = sample_labeled_data(data[tmp_idx], targets[tmp_idx], 
                         num_labels= num_prov + num_dum ,num_classes=10,index=None)
    lb_all_idx = tmp_idx[sorted(list(set(range(len(tmp_idx))) - set(ul_idx)))]
    ul_idx = tmp_idx[ul_idx.tolist()]

    #以下同様
    prov_data,prov_s,prov_idx = sample_labeled_data(data[ul_idx], targets[ul_idx], 
                         num_labels= num_prov  ,num_classes=10,index=None)
    dum_idx = ul_idx[list(set(range(len(ul_idx))) - set(prov_idx))]
    prov_idx = ul_idx[prov_idx.tolist()]
    
    lb_data,lb_s,lb_idx = sample_labeled_data(data[lb_all_idx], targets[lb_all_idx], 
                         num_labels= num_lb ,num_classes=10,index=None)
    val_idx = lb_all_idx[sorted(list(set(range(len(lb_all_idx))) - set(lb_idx)))]
    lb_idx = lb_all_idx[lb_idx.tolist()]

    os.makedirs(main_path+"data_index",exist_ok=True)
    np.save(main_path+"data_index/lb",lb_idx)
    np.save(main_path+"data_index/val",val_idx)
    np.save(main_path+"data_index/temp",prov_idx)
    np.save(main_path+"data_index/temp_all",prov_idx)
    np.save(main_path+"data_index/lb_all",lb_all_idx)
    np.save(main_path+"data_index/correct",prov_s)
    np.save(main_path+"data_index/dummy",dum_idx)
    #return tmp_idx,tmp_s,prov_idx,prov_s,lb_idx,lb_s,val_idx#,val_s

In [ ]:
#突然変異(個体内)
def mutate(individual,indpb, para_len):
  for i,_max in enumerate(para_len):
    if random.random() < indpb:
      temp = individual[i]
      individual[i] += random.randint(0,_max-1)
      individual[i] += 1
      individual[i] %= _max
  return individual,
 
#交叉
def mate(individuals, prob=1):
  for child1, child2 in zip(individuals[::2], individuals[1::2]):
      if random.random() < prob:
        tools.cxUniform(child1, child2, 0.5)


#突然変異　　mutpb:個体ごとの確率,mutpb2:遺伝子座ごとの確率
def mutation(offspring, mutpb, mutpb2,para_len):
  for mutant in offspring:
      if random.random() < mutpb:
        mutate(mutant, mutpb2,para_len)


#適応度計算
def one_eval(main_path,g,i,ind,seeds,dum):
  with open(main_path + "progress.csv", 'w') as f:
    writer = csv.writer(f)
    writer.writerow([g,i,"running"])

  #パラメータ再設定(別でまとめてもいいかも)
  args = get_args(main_path,g,i)
  args.resume = False
  args.lr =1e-2
  args.epochs = 20
  args.batch_size = 32
  args.test_times = 3
  fitness = []
  train_acc = []

  #必要インデックスの取得
  lb_idx = np.load(args.index_dir + "lb.npy")
  val_idx = np.load(args.index_dir + "val.npy")
  prov_idx = np.load(args.index_dir+"temp.npy")
  para_index = np.load(args.index_dir + "para.npy", allow_pickle=True)

  #個体をラベル集合に変換
  para_ind = [i[j] for i,j in zip(para_index,ind)]

  args.load_encoder_path = main_path + "pretrain/latest_model.pth"

  #適応度計算
  for j,seed in enumerate(seeds):
      args.seed = seed
      args.save_dir = main_path + "{:03}gene/fix/{:03}/{:03}/".format(g,i,j)
      os.makedirs(args.save_dir,exist_ok=True)
      train_ac, test_ac,last_ac = tuning_main(args, lb_idx, val_idx, prov_idx, para_ind, dum)
      fitness.append((test_ac+last_ac)/2) #過学習および学習不足の差異を減らすため
      train_acc.append(train_ac)


  total_fitness = np.average(fitness) 
  correct = np.load(args.index_dir + "correct.npy")
  real_acc = accuracy_score(correct,para_ind) 
  print("{:03}gene/{:03}ind::acc:{:03f}".format(g,i,total_fitness))
  del ind.fitness.values
  ind.fitness.values = total_fitness,

  with open(main_path +"{:03}gene/".format(g)+ gene_his,'a') as f:
    writer = csv.writer(f)
    writer.writerow([total_fitness, real_acc]+fitness+train_acc)
  with open(main_path + "progress.csv", 'w') as f:
    writer = csv.writer(f)
    writer.writerow([g,i,"fin"])

#個体生成
def creat_pop(main_path):

  #encoerがない場合生成
  if not os.path.exists(main_path+"pretrain/")::
      args = get_pretrain_args(main_path)
      args.batch_size = 216
      print(datetime.datetime.now())
      CL_main(args)

  #事前学習
  args = get_args(main_path,0,0)
  args.save_dir = main_path + pre_path
  args.is_save = True
  args.use_test = True
  args.ga_ = False
  lb_idx = np.load(args.index_dir + "lb_all.npy")
  _,__,___ = tuning_main(args, lb_idx, [], [], [], [])

  args.load_fc_path = main_path + pre_path + "-1/best_fc_model.pth"
  val_idx = np.load(args.index_dir + "val.npy")
  val_acc,_,pre_mat = eval_(args, val_idx )
  
  #予測値の低いラベルの足切り
  y = np.where(np.array(pre_mat) > 0.075)
  z = [[] for _ in range(IND_SIZE)]
  for y0,y1 in zip(y[0],y[1]):
    z[y0].append(y1)
  np.save(args.index_dir + 'para.npy', z)
  zz = np.array([len(i) for i in z])
  np.save(args.index_dir + 'para_len.npy',zz)

  cor_cnt = 0
  correct = np.load(args.index_dir + "correct.npy")
  for k,tar in zip(correct,z):
    if np.any(tar == k):
      cor_cnt += 1
  print(cor_cnt)
  print(np.average(zz),zz)

 #初期個体の生成
  creator.create("FitnessMax", base.Fitness, weights=(1.0,))
  creator.create("Individual", list, fitness=creator.FitnessMax)
  toolbox = base.Toolbox()

  pop = []
  for i in range(GENERATION_SIZE):
    base_ind = [random.randrange(i) for i in zz]
    ind = creator.Individual(copy.deepcopy(base_ind))
    #mutate(ind,ev_miss)
    pop.append(ind)
  return pop

In [ ]:
#ラベル付きデータをtrainとvalidに分割(データ数が少ないため現在未使用)
def split_lb(main_path,num_train):
  index_dir = main_path + "data_index/"
  lb_all_idx = np.load(index_dir + "lb_all.npy")
  dset = getattr(torchvision.datasets,"CIFAR10")
  dset = dset("./data", train=True)
  data, targets = np.array(dset.data), np.array(dset.targets)

  lb_all_data, lb_all_s, lb_all_idx = sample_labeled_data(data, targets,
                                               num_labels=len(lb_all_idx), num_classes=10, index=lb_all_idx)
  lb_data, lbs, lb_idx = sample_labeled_data(data[lb_all_idx], targets[lb_all_idx],
                                               num_labels=num_train, num_classes=10, index=None)
  val_idx = np.array(sorted(list(set(range(len(lb_all_idx))) - set(lb_idx))))
  lb_idx = lb_all_idx[lb_idx.tolist()]
  val_idx = lb_all_idx[val_idx.tolist()]
  
  np.save(index_dir+"lb",lb_idx) 
  np.save(index_dir+"val",val_idx)

##Main exe

In [ ]:
#GA本体
def main_GA():

  #現在の学習経過の取得::gene_cnt:世代数,num_fin_ind:探索中個体番号
  gene_cnt,num_fin_ind = 0,0
  isRunning = ""
  if  os.path.exists(main_path + "progress.csv")==True:
    with open(main_path + "progress.csv", 'r') as f:
      for i,row in enumerate(csv.reader(f, lineterminator='\n')):
        gene_cnt = int(row[0])
        num_fin_ind = int(row[1])
        isRunning = row[2]
  else:
    with open(main_path + "progress.csv",'w') as f:
      f.write("")

  if isRunning == "running":
    if os.path.exists(main_path + "{:03}gene/fix/{:03}".format(gene_cnt,num_fin_ind)):
      shutil.rmtree(main_path + "{:03}gene/fix/{:03}".format(gene_cnt,num_fin_ind))

  
  #個体設定
  creator.create("FitnessMax", base.Fitness, weights=(1.0,))
  creator.create("Individual", list, fitness=creator.FitnessMax)
 
  toolbox = base.Toolbox()
 
  #toolbox.register("attribute", random.randint, 0, max_Magnitude)
  #toolbox.register("individual", tools.initRepeat, creator.Individual,
  #                 toolbox.attribute, IND_SIZE)
  #toolbox.register("population", tools.initRepeat, list, toolbox.individual)
  #toolbox.register("mate", tools.cxTwoPoint)
  #toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.1)
  toolbox.register("select", tools.selTournament, tournsize=2)
 
 
  pop = []        #全個体のlist
  fitnesses = []  #適応度のlist
  #test_acc_lis = []

  if gene_cnt == 0 and num_fin_ind == 0 and isRunning!="running":

    #初期個体生成
    if not os.path.exists(main_path+"data_index/"):
        split_GADMdata(main_path,LABELED_SIZE,VALIDATION_SIZE,IND_SIZE,DUM_SIZE)
    #pop = toolbox.population(n=GENERATION_SIZE)
    pop = creat_pop(main_path)
    os.makedirs(main_path+"{:03}gene/".format(gene_cnt),exist_ok=True)
    os.makedirs(main_path+"{:03}gene/fix/".format(gene_cnt),exist_ok=True)

    _ind = np.empty((0,IND_SIZE), int)
    for ind in pop:
        _ind = np.append(_ind, np.array([ind]),axis=0)
    np.save(main_path+"{:03}gene/ind".format(gene_cnt),_ind)

    seed = np.array(random.sample(range(2**31 -1),num_trial))
    np.save(main_path+"{:03}gene/seed".format(gene_cnt),seed)

    #ダミーのラベル生成
    dum_tar = np.array(random.sample([i%10 for i in range(DUM_SIZE)],DUM_SIZE))
    np.save(main_path+"{:03}gene/dum".format(gene_cnt),dum_tar)

    #適応度計算
    for i,ind in enumerate(pop):
        one_eval(main_path,gene_cnt,i,ind,seed,dum_tar)

  elif isRunning=="running":
      #現世代の個体取得
      _ind = np.load(main_path + "{:03}gene/ind.npy".format(gene_cnt))
      for ind_ in _ind:
          ind = creator.Individual(ind_.tolist())
          pop.append(ind)

    #計算済みの適応度取得
    acc_lis = []
    if os.path.exists(main_path +"{:03}gene/".format(gene_cnt)+ gene_his):
        with open(main_path +"{:03}gene/".format(gene_cnt)+ gene_his,'r') as f:
            for row in csv.reader(f,lineterminator='\n'):
                acc_lis.append(float(row[0]))

    seed = np.load(main_path+"{:03}gene/seed.npy".format(gene_cnt))
    dum_tar = np.load(main_path+"{:03}gene/dum.npy".format(gene_cnt))

    #適応度計算および個体と紐づけ
    for i,ind in  enumerate(pop):
        if i < num_fin_ind:
          ind.fitness.values = acc_lis[i],
        else:
          one_eval(main_path,gene_cnt,i,ind,seed,dum_tar)

  g = gene_cnt

  for gene_cnt in range(g + 1, NGEN):
 
    print("***generation{}*****************".format(gene_cnt))
    #次世代の個体生成
    elite = tools.selBest(pop, k= elite_num)
    offspring = toolbox.select(pop, len(pop)- elite_num)
    offspring = list(map(toolbox.clone, offspring))
    mate(offspring)
    para_len = np.load(main_path + "data_index/para_len.npy")
    mutation(offspring, MUTPB, MUTPB2, para_len)
    offspring.extend(elite)
    pop[:] = offspring
    
    os.makedirs(main_path+"{:03}gene/".format(gene_cnt),exist_ok=True)
    os.makedirs(main_path+"{:03}gene/fix/".format(gene_cnt),exist_ok=True)
    
    _ind = np.empty((0,IND_SIZE), int)
    for ind in pop:
        _ind = np.append(_ind, np.array([ind]),axis=0)
    np.save(main_path+"{:03}gene/ind".format(gene_cnt),_ind)

    seed = np.array(random.sample(range(2**31 -1),num_trial))
    np.save(main_path+"{:03}gene/seed".format(gene_cnt),seed)

    dum_tar = np.array(random.sample([i%10 for i in range(DUM_SIZE)],DUM_SIZE))
    np.save(main_path+"{:03}gene/dum".format(gene_cnt),dum_tar)
    
    #適応度計算
    for i,ind in enumerate(pop):
        one_eval(main_path,gene_cnt,i,ind,seed,dum_tar)

  return pop
 
 
if __name__ == "__main__":
  main_path = "/content/drive/My Drive/CLGA/"
  os.makedirs(main_path,exist_ok=True)
  print(main_GA())

In [ ]:
"""
path = '/content/drive/My Drive/CLGA/'
with open(path + "progress.csv", 'w') as f:
    writer = csv.writer(f)
    writer.writerow([0,0,"running"])
"""

#Research

In [ ]:
"""
個体集計
genes に二次配列で個体を取得
"""
main_path = "/content/drive/My Drive/CLGA/"
correct = np.load(main_path + "data_index/correct.npy").tolist()
para_index = np.load(args.index_dir + "para.npy", allow_pickle=True).tolist()
genes = []
gene_tmp = []
with open(main_path + "progress.csv", 'r') as f:
    for i,row in enumerate(csv.reader(f, lineterminator='\n')):
        gene_cnt = int(row[0])
        num_fin_ind = int(row[1])
        isRunning = row[2]
num_gene = gene_cnt - 1

for i in range(num_gene):
  gene_tmp = []
  x = np.load(main_path + "{:03}gene/ind.npy".format(i)).tolist()
  x = [i[j] for i,j in zip(para_index,x)]
  genes.append(x)
  print("\r{}/{}".format(i,num_gene),end="")
  time.sleep(.10)

print("")
num_ind = len(genes[0][0])
print(num_gene, num_ind)
print(genes)

In [ ]:
"""
x :　個体の適応度
y :  個体の実際の精度
"""

gene_his = "gene_his.csv"
x = []
y = []
for i in  range(num_gene):
  tmp1 = []
  tmp2 = []
  with open(main_path +"{:03}gene/".format(i)+ gene_his,'r') as f:
    for row in csv.reader(f,lineterminator='\n'):
      tmp1.append(float(row[0]))
      tmp2.append(float(row[1]))
  x.append(tmp1)
  y.append(tmp2)
  print("\r{}/{}".format(i,num_gene),end="")

print("")
max_index = [i.index(max(i)) for i in x]
fit_max = [i[j] for i,j in zip(x,max_index)]
corret_for_fitmax = [i[j] for i,j in zip(y,max_index)]

In [ ]:
def ensemble(targets,num_labels,bias=None):
  targets = np.array(targets)
  num = targets.shape[1]
  if bias==None:
    bias = [1.0 for _ in range(num)]
  ans_mat = np.array([[0.0 for __ in  range(num_labels)] for _ in  range(num)])
  for j,target in enumerate(targets):
    for i,x in enumerate(target):
      ans_mat[i][int(x)] += bias[j]
  ans = [np.argmax(x) for x in ans_mat]
  return ans


In [ ]:
"""
acc_lis : 個体の適応度
ens_acc_lis : 1世代におけるあるデータにおける
              もっとも採択されたラベルに対する精度
"""

acc_lis = []
ens_acc_lis = []
for ones in genes:
  tmp=[]
  for ind in ones:
    cnt = 0
    for i,j in zip(ind,correct):
      if i==j:
        cnt += 1
    tmp.append(cnt)
  ens = ensemble(ones,10)
  acc = accuracy_score(ens,correct)
  ens_acc_lis.append(acc)
  acc_lis.append(tmp)

print(ens_acc_lis)

In [ ]:
#散布図

import matplotlib.cm as cm
x_ = []
y_ = []
num1 = 0
rang = 10000
for xx,yy in zip(x,y):
  x_.extend(xx)
  y_.extend(yy)
print(len(x_))
print(np.corrcoef(x_,y_)[0][1])


fig = plt.figure()
plt.scatter(y_[num1:num1 + rang],x_[num1:num1 + rang])

plt.ylabel("fitness value" ,fontsize=15)
plt.xlabel("correct num", fontsize=15)
plt.grid(True)
fig.savefig("img.png")

In [ ]:
#世代に対する推移

import glob
import matplotlib.pyplot as plt
from statistics import mean

max_acc = []
acc_lis = []

max_acc = [ max(x_) for x_ in y]
ave_acc = [mean(li) for li in y]
ca_min = min(ave_acc) - 0.05
ca_max = max(max_acc) + 0.05

acc_lis = tuple(x)
al_max = max([max(i) for i in acc_lis]) + 0.05
al_min = min([min(i) for i in  acc_lis]) - 0.05
#print(max_acc)
fig, ax = plt.subplots(figsize=(10,5),dpi=100)
ax2 = ax.twinx()
num = 20


#fig = plt.figure(figsize=(10,10),dpi=200)

bp = ax.boxplot(acc_lis, positions=[i for i in range(num_gene)])
#ax.set_xticklabels([str(i) for i in range(1,len(acc_lis)+1)])
ax.set_ylim([al_min , al_max ])

ax2.set_xticks(np.arange(-1, num_gene, 10))
ax2.set_xticklabels([str(i) for i in range(0,num_gene,10)])
ax2.set_xticks(np.arange(-1, num, 10.0),minor=True)
ax2.set_ylim([ca_min, ca_max])

ax2.plot(max_acc,marker='.', label='best correct rate')
ax2.plot(ave_acc,marker='.', label='average correct rate')

ax2.legend()

plt.legend(loc='lower right', fontsize=15)
plt.grid()
ax.set_xlabel('generation', fontsize=15)
ax2.set_ylabel('accuracy about individual', fontsize=15)
ax.set_ylabel('fitness accuracy ', fontsize=15)

plt.savefig("graph.png")
plt.show()

#SomeThing

In [ ]:
#TDGA()
    off_o = list(map(toolbox.clone, toolbox.select(pop, GENERATION_SIZE)))
    off_p = list(map(toolbox.clone, pop))

    mate(off_o)
    para_len = np.load(main_path + 'data_index/para_len.npy')
    mutation(off_o,MUTPB, MUTPB2, para_len)
    mutation(off_p,MUTPB, MUTPB2, para_len)

    off_d = list(map(toolbox.clone, elite))
    off_d.extend(off_o)
    off_d.extend(off_p)

    tt = gene_cnt/ NGEN
    temperature = pow(t_init, 1-tt)*pow(t_fin, tt)
    def closure(pop_):
      n = len(pop_) + 1
      npop = np.array(pop_)
      cnt_list = np.array([np.count_nonzero(npop == i, axis=0) for i in range(10)]).T.tolist()
      prov_h = [[-x / n * math.log2(x / n + eps ) for x in lis]for lis in cnt_list]

      prov_hall =sum([sum(x) for x in prov_h])
      E_sum = sum([ind.fitness.values[0] for ind in pop_])
      def compute_F(ind):
        h = prov_hall
        for i,gene in enumerate(ind):
          h -= prov_h[i][gene]
          num_g = cnt_list[i][gene] + 1
          h += -num_g / n * math.log2(num_g / n)


        F = (E_sum + ind.fitness.values[0])/n - h* temperature
        return F
      return compute_F 

    selected = elite
    for i in range(GENERATION_SIZE - elite_num):

      f = closure(selected)
      Fs = [f(ind) for ind in off_d]
      idx = Fs.index(min(Fs))
      selected.append(off_d[idx])
      off_d.pop(idx)